In [1]:
install.packages("pacman")

pacman::p_load(
    ISLR, #For the dataset Carseats
    magrittr,
    caret,
    rpart,
    dplyr
)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


ISLR installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’



caret installed



In [2]:
# Sample data (replace this with your actual dataset)
data <- Carseats %>%
  select(Sales, CompPrice, Income, Advertising, Population, Price)

# Generate the correlation matrix
cor_matrix <- cor(data)

# Print the correlation matrix
print(cor_matrix)


                  Sales   CompPrice       Income Advertising   Population
Sales        1.00000000  0.06407873  0.151950979  0.26950678  0.050470984
CompPrice    0.06407873  1.00000000 -0.080653423 -0.02419879 -0.094706516
Income       0.15195098 -0.08065342  1.000000000  0.05899471 -0.007876994
Advertising  0.26950678 -0.02419879  0.058994706  1.00000000  0.265652145
Population   0.05047098 -0.09470652 -0.007876994  0.26565215  1.000000000
Price       -0.44495073  0.58484777 -0.056698202  0.04453687 -0.012143620
                  Price
Sales       -0.44495073
CompPrice    0.58484777
Income      -0.05669820
Advertising  0.04453687
Population  -0.01214362
Price        1.00000000


In [28]:
data <- Carseats

set.seed(123)  # For reproducibility
trainIndex <- createDataPartition(data$Price, p = 0.8, list = FALSE)
train_data <- data[trainIndex, ]
test_data <- data[-trainIndex, ]

In [29]:
# Here, we'll tune the cp (complexity parameter) hyperparameter of the decision tree
cp_grid <- expand.grid(.cp = seq(0.001, 0.1, by = 0.001))

In [27]:
ctrl <- trainControl(method = "cv", number = 5, verboseIter = TRUE)

# Train the model with the hyperparameter grid and cross-validation
dt_model <- train(
  Price ~ CompPrice,
  data = train_data,
  method = "rpart",
  trControl = ctrl,
  tuneGrid = cp_grid
)


+ Fold1: cp=0.001 
- Fold1: cp=0.001 
+ Fold2: cp=0.001 
- Fold2: cp=0.001 
+ Fold3: cp=0.001 
- Fold3: cp=0.001 
+ Fold4: cp=0.001 
- Fold4: cp=0.001 
+ Fold5: cp=0.001 
- Fold5: cp=0.001 
Aggregating results
Selecting tuning parameters
Fitting cp = 0.005 on full training set


In [32]:
best_hyperparameters <- dt_model$bestTune
final_dt_model <- rpart(Price ~ CompPrice, data = train_data,
                     cp = best_hyperparameters$cp)

In [33]:
# Extract the actual target values from the test data
actual_values <- test_data$Price

# Make predictions on the test data using the final model
predictions <- predict(final_dt_model, newdata = test_data)

# Calculate Root Mean Squared Error (RMSE)
rmse <- sqrt(mean((actual_values - predictions)^2))
print(paste("Root Mean Squared Error (RMSE):", rmse))

# Calculate R-squared (R2) Score
r_squared <- cor(predictions, actual_values)^2
print(paste("R-squared (R2) Score:", r_squared))

[1] "Root Mean Squared Error (RMSE): 18.6594923360671"
[1] "R-squared (R2) Score: 0.298083953390265"
